In [1]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: langchain-anthropic
    Found existing installation: langchain-anthropic 0.1.16
    Uninstalling langchain-anthropic-0.1.16:
      Successfully uninstalled langchain-anthropic-0.1.16
Note: you may need to restart the kernel to use updated packages.


In [6]:
from dotenv import load_dotenv
_ = load_dotenv()

In [9]:
import os

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://world-weather.info/forecast/usa/san_francisco/june-2024/', 'content': 'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}, {'url': 'https://www.accuweather.com/en/us/san-francisco/94103/june-weather/347629', 'content': 'Get the monthly weather forecast for San Francisco, CA, including daily high/low, historical averages, to help you plan ahead.'}]


In [11]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [12]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1000)

In [16]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [17]:
model_with_tools = model.bind_tools(tools)



In [33]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [34]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_RHIhRGo9bVJysc1UQIc3xIR7'}]


In [35]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)


In [36]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='8b829351-3ef6-4182-8d88-0874e594935a'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-28d134df-2902-48c4-9204-a1373f496173-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [37]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='189ef703-e331-4452-ac11-ebe584b42d43'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rj0md1N32JWN3qQXoVwu6nZm', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-40787bcf-536a-48ca-8629-088408cdb823-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_rj0md1N32JWN3qQXoVwu6nZm'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109}),
 ToolMessage(content='[{"url": "https://www.accuweather.com/en/us/san-francisco/94103/june-weather/347629", "content": "Get the monthly weather forecast for San Francisco, CA, including daily high

In [38]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TFGXYOCgGxks2555XTreAbP7', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-60076aac-c6c9-45cb-bb95-ff0a0647a80a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_TFGXYOCgGxks2555XTreAbP7'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.timeanddate.com/weather/usa/san-francisco/hourly", "content": "Hour-by-Hour Forecast for San Francisco, California, USA. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Clim

In [39]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="")
  

The current weather in San Francisco is partly cloudy with a temperature of 57.0°F (13.9°C). The wind is blowing from the north at 4.3 mph (6.8 kph), and the humidity is at 89%. The weather conditions are expected to be partly cloudy. For more detailed information, you can check the [Weather Underground](https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-6-29) or [Weather API](https://www.weatherapi.com/) websites.

In [41]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [42]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [45]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")
    

{'agent': {'messages': [AIMessage(content="Hello Bob! It seems like you're introducing yourself again. How can I assist you today, Bob?", response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 133, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1997ae9a-1a7c-479c-b53f-97cecf3e47d9-0', usage_metadata={'input_tokens': 133, 'output_tokens': 22, 'total_tokens': 155})]}}
----


In [46]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob. How can I assist you, Bob?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 167, 'total_tokens': 181}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-be0dbc38-3e58-436a-b223-36019fac989e-0', usage_metadata={'input_tokens': 167, 'output_tokens': 14, 'total_tokens': 181})]}}
----


In [47]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal information like your name. If there's anything else you'd like to know or discuss, feel free to ask!", response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 86, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-14ff7ce1-d433-4fac-ab70-61fe9a41e50e-0', usage_metadata={'input_tokens': 86, 'output_tokens': 31, 'total_tokens': 117})]}}
----
